In [ ]:
import torch
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

DATA_DIR = "/home/borth/2d-gaussian-splatting/data/DTU"
SCAN_ID = "scan37"


# Camera Data

In [ ]:
cameras = np.load(Path(DATA_DIR, SCAN_ID, "cameras.npz"))
print(list(cameras.keys()))

In [ ]:
ply_path = Path(DATA_DIR, SCAN_ID, "points.ply")
visualize_scene(ply_path)

In [ ]:
path = "/home/borth/2d-gaussian-splatting/output/input.ply"
visualize_scene(ply_path)

In [ ]:
import torch

idx = 0
depth_map = torch.load(Path(DATA_DIR, SCAN_ID, f"depths/{idx:04}.pt"), weights_only=False)
print(depth_map.keys())
print(depth_map["depth"], depth_map["depth"].shape)
print(depth_map["coord"], depth_map["coord"].shape)
print(depth_map["error"], depth_map["error"].shape)
print(depth_map["weight"], depth_map["weight"].shape)

In [ ]:
from PIL import Image
img = Image.open("/home/borth/2d-gaussian-splatting/data/DTU/scan24/mask/000.png")
img

In [ ]:
from PIL import Image
img = Image.open("/home/borth/2d-gaussian-splatting/data/DTU/scan24/images/0000.png")
np.asarray(img).shape

In [ ]:
import trimesh
import open3d as o3d

# Load the mesh with trimesh
mesh = trimesh.load_mesh("/home/borth/2d-gaussian-splatting/output/11-02-25/scan55/train/ours_30000/fuse_post.ply")

# Convert trimesh to open3d format
o3d_mesh = o3d.geometry.TriangleMesh()
o3d_mesh.vertices = o3d.utility.Vector3dVector(mesh.vertices)
o3d_mesh.triangles = o3d.utility.Vector3iVector(mesh.faces)

# Compute normals for better visualization
o3d_mesh.compute_vertex_normals()

# Visualize with open3d
o3d.visualization.draw_plotly([o3d_mesh])


In [4]:
import trimesh

# Load the mesh from a file
mesh = trimesh.load_mesh("/home/borth/2d-gaussian-splatting/logs/2025-02-13/10-15-18/train/ours_10/fuse_cull.ply")

import pyrender
# Convert to a Pyrender-compatible format
mesh_pyrender = pyrender.Mesh.from_trimesh(mesh)

# Create a scene
scene = pyrender.Scene()
scene.add(mesh_pyrender)

# Add better lighting (Directional Light)
light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=3.0)
scene.add(light)

# Add a camera for a better view
camera = pyrender.PerspectiveCamera(yfov=0.7)
scene.add(camera, pose=[[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 1, 2],  # Move the camera back to see the model
                        [0, 0, 0, 1]])

# Add a camera for better viewing
camera = pyrender.PerspectiveCamera(yfov=0.7)
scene.add(camera, pose=[[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 1, 2],  # Move camera back to view model
                        [0, 0, 0, 1]])

# Render interactively
pyrender.Viewer(scene, use_raymond_lighting=True)  # Opens an interactive window

NoSuchDisplayException: Cannot connect to "None"